Using this to write out my resume, then copying it. 

Would be a fun project to do this automatically, now that I have all the styling etc. for it :) 

# Daniel McCann-Sayles' Portfolio

### Summary

<p style="display: flex; justify-content: space-between; right-margin: 50px ">
  <span>Cisco</span>
  <span ">thing2</span>
</p>

Summary :p

### Work Experience

Cisco - Intern

### Projects

##### Notebook portfolio

ongoing...

##### Kid Fight Game

##### MACM exploration

##### AI Generation

##### Speech to LLM App


I wanted to take better notes/ have a personal AI assistant.  
I worked on this for a couple of weeks in July while I had some free time. 

I like to talk to myself when walking, and I would like to be able to brainstorm ideas. 
Ideally I could offload some cognitive tasks, such as recalling or looking things up. I also find that in a back and forth manner I can come up with better ideas.

To do this I made a very simple iOS app that sends audio to a node server that transcribes the audio and listens for a wake word. Upon this wake word a call to ChatGPT is triggered with the past context. The iOS app receives this audio.

All the audio is also transcribed and kept as notes. I also explored using calls to ChatGPT to clean up this audio, though this naive approach didn’t work super well. 

Some things I considered doing but didn’t get around to were:
1. Adding keywords that could create new notes, or give the context of the previous x lines of notes to ChatGPT, along with a command to format it in some way.
2. More generally, giving more options to ChatGPT - e.g. searching, or custom function calling. 

Takeaways:
1. I do not enjoy Swift. I was able to hack together an app using a lot of ChatGPT and debugging. 
    
    > Tech stacks are just *tools* that allow you to do *things*. Doing *things* is what’s important. 
2. I struggled with audio transcription - I tend to mumble and the transcriber I was using did not like that.
3. I didn’t have a measurable goal - 
    1. I stopped working on this because it wasn’t giving validation
    2. It’s important to build in measurable outcomes to prototype stages - how can I validate if this idea is *worth continuing*? What value can I derive from *each stage*



##### Cisco Hackathon

I competed in an internal Cisco hackathon to explore how we could improve  customer experience use cases with AI. 

My team won 1st place out of over 20 teams!

The compeotition took place over the span of two weeks. It was hosted by a different organization in Cisco, so it was different products and a different setting than I'm used to. Luckily my team had some experience we were able to draw on.

I had a lot of fun meeting people, collaborating, and creating our project. 


I played a few key roles on the team: 
1. Collaborating closely with our team lead to hash out his ideas and customer knowledge, and come up with a winning idea. He has a lot of knowledge around the specific field; *together* we were able to turn this into an implementation. 
2. Executing on this; I quickly learned how to build, and built, a smart AI assistant using Streamlit and a RAG system with FAISS and LangChain.

This showcases my holistic skill set. <span id="text-accent">I’m at my best when I have the ability to work closely with the product, with stakeholders, and with the implementation itself!</span>

My team lead graciously said this about me:

> "Daniel continuously demonstrated his ability to navigate high-level business strategy and translate that into ideas, concepts, code, and connections that support the achievement of those goals."

** add some more about this showing my ideation, prototyping, and ability to see the big picture, while also coding it! **
